In [211]:
import sys
import os
import numpy as np
import xarray as xr
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.metrics import brier_score_loss
from sklearn.linear_model import LogisticRegression

sys.path.append(os.path.expanduser('~/Documents/Hybrid/'))
from Hybrid.neuralnet import construct_modeldev_model, earlystop, BrierScore, ConstructorAndCompiler
#from Hybrid.optimization import multi_fit_multi_eval, multi_fit_single_eval, ranked_prob_score
from Hybrid.dataprep import test_trainval_split, multiclass_log_forecastprob, singleclass_regression, multiclass_logistic_regression_coefficients, scale_other_features
from Hybrid.interpretation import combine_input_output, composite_extremes

crossval = True # Only for the trainvalsplit.
nfolds = 3
balanced = True
focus_class = -1 # Index of the class to be scored and benchmarked through bss

name = 'tg-ex-q0.75-21D_ge7D_sep12-15'
#name = 'tg-ex-q0.75-21D_ge7D_sep19-21'
name2 = 'tg-anom_JJA_45r1_31D-roll-mean_sep12-15'

#predictor_dir = Path('/nobackup/users/straaten/predsets/objective_balanced_cv/no_pdomjo/') # Can change to the objectively selected sets
predictor_dir = Path('/nobackup/users/straaten/predsets/objective_balanced_cv/') # Can change to the objectively selected sets
for_obs_dir = Path('/nobackup/users/straaten/predsets/full/') 

predictors = pd.read_hdf(predictor_dir / f'{name2}_multi_d15_b3_predictors.h5', key = 'input').iloc[:,:7]
forc = pd.read_hdf(for_obs_dir / f'{name}_forc.h5', key = 'input')
obs= pd.read_hdf(for_obs_dir / f'{name}_obs.h5', key = 'target')

In [212]:
# Preparing only the trainval set.
X_test, X_trainval, generator = test_trainval_split(predictors, crossval = crossval, nfolds = nfolds, balanced = balanced)
forc_test, forc_trainval, generator = test_trainval_split(forc, crossval = crossval, nfolds = nfolds, balanced = balanced)
obs_test, obs_trainval, generator = test_trainval_split(obs, crossval = crossval, nfolds = nfolds, balanced = balanced)

feature_input, feature_scaler = scale_other_features(X_trainval)
raw_predictions = multiclass_log_forecastprob(forc_trainval)
obs_input = obs_trainval.values

fitting a new feature scaler


In [213]:
predictors

variable              sst_nhplus                     siconc_nhmin sst_nhplus  \
timeagg                       21        15        7            31         11   
clustid                        4         1         0            1          2   
metric                   spatcov   spatcov   spatcov      spatcov    spatcov   
time       separation                                                          
1998-06-19 12          -0.051529  0.030985 -0.000787     0.002585  -0.056700   
1998-06-20 13          -0.050145  0.031957 -0.000919     0.002649  -0.056194   
1998-06-21 14          -0.049024  0.032371 -0.000980     0.002755  -0.056300   
1998-06-22 15          -0.048194  0.032323 -0.000972     0.002891  -0.057027   
1998-06-23 12          -0.047589  0.031867 -0.000904     0.003043  -0.058528   
...                          ...       ...       ...          ...        ...   
2019-06-21 15          -0.009250 -0.022996 -0.000522    -0.005122   0.017135   
2019-06-22 12          -0.009223 -0.023906 -0.000518    -0.005416   0.017303   
2019-06-23 13          -0.009382 -0.024418 -0.000467    -0.005723   0.018011   
2019-06-24 14          -0.009428 -0.024719 -0.000381    -0.006028   0.019147   
2019-06-25 15          -0.009570 -0.025504 -0.000348    -0.006298   0.020860   

variable              transp_europe swvl13_europe  
timeagg                          31            11  
clustid                           1             0  
metric                      spatcov       spatcov  
time       separation                              
1998-06-19 12             -0.000023      0.000439  
1998-06-20 13             -0.000028      0.000433  
1998-06-21 14             -0.000032      0.000428  
1998-06-22 15             -0.000036      0.000427  
1998-06-23 12             -0.000039      0.000419  
...                             ...           ...  
2019-06-21 15              0.000038     -0.000095  
2019-06-22 12              0.000036     -0.000121  
2019-06-23 13              0.000033     -0.000160  
2019-06-24 14              0.000033     -0.000205  
2019-06-25 15              0.000032     -0.000255  

[2272 rows x 7 columns]

In [214]:
# Setting up the model
construct_kwargs = dict(n_classes = obs_trainval.shape[-1],
        n_hidden_layers= 1,
        n_features = feature_input.shape[-1],
        n_hiddenlayer_nodes = 4)

compile_kwargs = dict(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics = ['accuracy',BrierScore(class_index = focus_class)])

constructor = ConstructorAndCompiler(construct_modeldev_model, construct_kwargs, compile_kwargs)

fit_kwargs = dict(batch_size = 32, epochs = 200, shuffle = True, callbacks = [earlystop(patience = 7, monitor = 'val_loss')])

In [215]:
# Training the model
model = constructor.fresh_model()
model.fit(x = [feature_input, raw_predictions], y=obs_input, validation_split = 0.4, **fit_kwargs)

Epoch 1/200
36/36 [==============================] - 0s 3ms/step - loss: 0.5896 - accuracy: 0.7014 - brier: 0.1994 - val_loss: 0.6794 - val_accuracy: 0.6328 - val_brier: 0.2330
Epoch 2/200
36/36 [==============================] - 0s 1ms/step - loss: 0.5886 - accuracy: 0.7014 - brier: 0.1988 - val_loss: 0.6758 - val_accuracy: 0.6328 - val_brier: 0.2311
Epoch 3/200
36/36 [==============================] - 0s 1ms/step - loss: 0.5868 - accuracy: 0.7014 - brier: 0.1980 - val_loss: 0.6755 - val_accuracy: 0.6328 - val_brier: 0.2310
Epoch 4/200
36/36 [==============================] - 0s 1ms/step - loss: 0.5840 - accuracy: 0.7014 - brier: 0.1969 - val_loss: 0.6746 - val_accuracy: 0.6328 - val_brier: 0.2307
Epoch 5/200
36/36 [==============================] - 0s 1ms/step - loss: 0.5800 - accuracy: 0.7014 - brier: 0.1951 - val_loss: 0.6725 - val_accuracy: 0.6328 - val_brier: 0.2297
Epoch 6/200
36/36 [==============================] - 0s 1ms/step - loss: 0.5741 - accuracy: 0.7049 - brier: 0.1929 

In [216]:
# Test the predictions
test_features, _ = scale_other_features(X_test, fitted_scaler=feature_scaler)
test_forcs = multiclass_log_forecastprob(forc_test)
test_preds = model.predict(x = [test_features, test_forcs])

using a pre-fitted feature scaler


In [217]:
model.evaluate(x = [test_features, test_forcs], y = obs_test.values)

11/11 [==============================] - 0s 638us/step - loss: 0.6052 - accuracy: 0.7074 - brier: 0.2008


[0.6051961183547974, 0.7073863744735718, 0.2008126676082611]

In [218]:
np.mean((obs_test.iloc[:,focus_class] - forc_test.iloc[:,focus_class])**2)

0.23460872275046382

In [38]:
np.mean((obs_test.iloc[:,focus_class] - test_preds[:,focus_class])**2)

0.202847